# Multiclass Classification with MNIST
This time the purpose is to do multiclass clasification of MNIST dataset.
The project is has the following steps:
1) Load MNIST dataset ;
2) Split MNIST to pandas.DataFrame of pictures and pandas.Series of labels;
3) Check what is the data structure;
4) Convert ndarrays to PIL.Images to see what are the objects;
5) Split datasets to training and testing sets using Scikit-Learn module;
6) Train models: SGDClassifier, SoftMax Regression, SVC, NaiveBayesian Model, Random Forest Classifier;
7) Use f1_score, cross value score (accuracy) and confusion matrix to validate model;
8) Get conclusion and select the best model.

First we load necessary modules not Scikit-Learn related:

In [17]:
from pandas import DataFrame
from numpy import array
from PIL import Image
from typing import Union


Then function from the Scikit-Learn module taht allows to fetch MNIST dataset

In [1]:
from sklearn.datasets import fetch_openml

And finally the other fuctions and models from  Scikit-Learn:

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, recall_score, precision_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
mnist = fetch_openml('mnist_784', version=1)

Get set of data and labels. As at this point I do not know what data type I should expect, I check the type of both objects.

In [4]:
X, y = mnist.data, mnist.target
print(type(X), type(y))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


Knowing we are handing with pandas: DataFrame and Series, lets check data structure:

In [22]:
print(X.head(2)) # one picture consists of 784 pixels: rows == pictures, columns == pixels (784=28^2 hence 28 rows and 28 cols)
print(y.head(2)) # classification: what number is described by a row in X

   pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   

   pixel10  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0        0  ...         0         0         0         0         0         0   
1        0  ...         0         0         0         0         0         0   

   pixel781  pixel782  pixel783  pixel784  
0         0         0         0         0  
1         0         0         0         0  

[2 rows x 784 columns]
0    5
1    0
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']


Elements of `X` should represent images of numbers `0,...,9`.
Let us see these by converting `ndarrays` to `PIL.Image`. 
For this we create the converting function.

In [5]:
def array_to_pillow_image(arr: array, width: int = 28, height: int = 28, show: bool = True) -> Image:
    if not len(arr) == width * height:
        raise ValueError("Number of pixels should be equal to width * height!")
        
    image = Image.new('RGB', (width, height), color='white')
    for i in range(len(arr)):
            image.putpixel((i % 28, i // 28), (arr[i], 0 , 0))

    if show:
        image.show()
        
    return image


Use the function created above to see the image of the first element (which is `0`):

In [42]:
array_to_pillow_image(array(X.loc[1]))


Now let us split the set of data and labels training and testing set
(we do it by spliting the set of indices, later we select apopriate data).

In [6]:
train_set_indices: list[int]
test_set_indices: list[int]
train_set_indices, test_set_indices = train_test_split(list(range(len(X))), test_size=0.2, random_state=42)

Having indices splitted, we do data selection to training and testing sets:

In [12]:
X_train: DataFrame = X.loc[X.index.isin(train_set_indices)]
y_train: DataFrame = y.loc[y.index.isin(train_set_indices)]
X_test: DataFrame = X.loc[X.index.isin(test_set_indices)]
y_test: DataFrame = y.loc[y.index.isin(test_set_indices)]

At some point I thought it would be nice to know how much time it takes to run certain method of preselected models. It can be done in a nice way
provided you modify the class definition itself (so we cannot do that this way here, but present this metaclass anyway).

In [47]:
class TimeDecorer(type):
    def __new__(cls, clsname, clsbase, clsdict):
        new_cls_dict: dict = {key: val if not callable(val) else time_decor(val) for key, val in clsdict.items()}
        return super(cls, TimeDecorer).__new__(cls, clsname, clsbase, new_cls_dict)

Now we train the preselected models:

In [13]:
sgdc: SGDClassifier = SGDClassifier()
sgdc.fit(X_train, y_train)

SGDClassifier()

In [16]:
nbc: MultinomialNB = MultinomialNB()
nbc.fit(X_train, y_train)

MultinomialNB()

In [18]:
rfc: RandomForestClassifier = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
svc: SVC = SVC()
svc.fit(X_train, y_train)

SVC()

In [49]:
smc: LogisticRegression = LogisticRegression(multi_class='multinomial', solver='lbfgs')
smc.fit(X_train, y_train)
                        

C:\Users\Daric\anaconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Daric\anaconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

And now we do predictions on testing set and we validate the models accuracy.

In [52]:
sgdc_predictions = sgdc.predict(X_test)
sgdc_cvs = cross_val_score(sgdc, X_train, y_train, cv=3, scoring='accuracy')
print(sgdc_cvs)
print(confusion_matrix(sgdc_predictions, y_test))
print(f1_score(sgdc_predictions, y_test, average='macro'))

[0.86907377 0.82803879 0.86226294]
[[1291    1    4    8    1    8    5    7   15    7]
 [   1 1547   13    8    3    8    3    1   22    5]
 [  14   12 1224   80    5   10   12   22   38    6]
 [   0    7   10 1145    4   23    0    1   35   12]
 [   3    2   28    1 1165   22   15   11   17   39]
 [  12   17   17  115   15 1142   19   10  143   33]
 [  16    1   27   13   11   28 1338    0   22    2]
 [   2    4   18   18   10    3    1 1410   13  108]
 [   3    8   28   26   10   19    3    1 1002    7]
 [   1    1   11   19   71   10    0   40   50 1201]]
0.8882297938126686


In [53]:
nbc_predictions = nbc.predict(X_test)
nbc_cvs = cross_val_score(nbc, X_train, y_train, cv=3, scoring='accuracy')
print(nbc_cvs)
print(confusion_matrix(nbc_predictions, y_test))
print(f1_score(nbc_predictions, y_test, average='macro'))

[0.81882466 0.81877109 0.83794064]
[[1215    0   13    5    5   34   16   13    4    6]
 [   1 1498   24   24    2   14   32   24   77   15]
 [   8    6 1139   69    8   10   19   10   22    9]
 [   4   13   28 1175    4  161    1    5   62   12]
 [   2    1   17    2  957   29   12   25   12   87]
 [  21   10    4   23    5  854   26    0   49    6]
 [  29    5   62   11   30   28 1277    1   14    0]
 [   1    1   11   12    5    4    0 1243    6   42]
 [  60   63   78   63   44  104   13   47 1058   53]
 [   2    3    4   49  235   35    0  135   53 1190]]
0.8273553956195601


In [54]:
rfc_predictions = rfc.predict(X_test)
rfc_cvs = cross_val_score(rfc, X_train, y_train, cv=3, scoring='accuracy')
print(rfc_cvs)
print(confusion_matrix(rfc_predictions, y_test))
print(f1_score(rfc_predictions, y_test, average='macro'))

[0.96400064 0.96046499 0.96732026]
[[1325    0    2    1    2    3    4    2    0    6]
 [   0 1576    3    0    0    2    1    7    8    7]
 [   4    5 1338   24    4    4    1   20   12    4]
 [   0    9    2 1357    0   16    0    0   14   18]
 [   2    2    6    1 1250    2    3    5    5   21]
 [   2    0    2   14    0 1225    7    0    8    3]
 [   2    0    8    1    5    7 1379    0    6    1]
 [   1    5    8   12    3    2    0 1448    6   12]
 [   7    3    9   13    2   10    1    3 1291    8]
 [   0    0    2   10   29    2    0   18    7 1340]]
0.966181227372992


In [55]:
svc_predictions = svc.predict(X_test)
svc_cvs = cross_val_score(svc, X_train, y_train, cv=3, scoring='accuracy')
print(svc_cvs)
print(confusion_matrix(svc_predictions, y_test))
print(f1_score(svc_predictions, y_test, average='macro'))

[0.97546472 0.97262549 0.97658845]
[[1329    0    3    0    1    0    1    1    2    5]
 [   1 1585    4    2    0    1    0    6    6    8]
 [   3    4 1348   11    2    2    0   12    8    2]
 [   0    3    2 1386    0   15    0    1   12    8]
 [   1    2    3    2 1269    2    4    5    4   14]
 [   2    0    2   11    0 1236    4    1    9    2]
 [   2    0    4    1    3    9 1384    0    6    0]
 [   1    4    8    9    2    1    0 1465    4   11]
 [   4    2    5    7    2    7    3    1 1303    5]
 [   0    0    1    4   16    0    0   11    3 1365]]
0.9762962373195112


In [56]:
smc_predictions = smc.predict(X_test)
smc_cvs = cross_val_score(smc, X_train, y_train, cv=3, scoring='accuracy')
print(smc_cvs)
print(confusion_matrix(smc_predictions, y_test))
print(f1_score(smc_predictions, y_test, average='macro'))

C:\Users\Daric\anaconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Daric\anaconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Daric\anaconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 

[0.91407296 0.91412653 0.92215793]
[[1289    0    4    6    4    7    7    5   11    7]
 [   1 1557   20    7    0   11    3    4   24   10]
 [   6    5 1229   30    8    8   20   28   17    7]
 [   0    9   25 1298    5   53    0    4   38   14]
 [   5    2   16    1 1194   16   14    8    7   40]
 [  14    7    8   38    4 1087   14    5   39    5]
 [  12    0   19    4   13   20 1332    0   12    0]
 [   6    3   15   14    6    2    2 1408   10   45]
 [   8   15   35   19   12   54    4    2 1186   22]
 [   2    2    9   16   49   15    0   39   13 1270]]
0.9166119578810752


It cleary follows from the comparison of the Cross Value Score and F1 score that `SVC` produces the best predictions, a bit behind is `Random Forest Classifier`. Other models produce also decent results but these are declassified by results of `SVC` and `Random Forest Classifier`.